In [ ]:
%matplotlib inline
import os
from bs4 import BeautifulSoup
import pandas as pd
import random
import matplotlib.pyplot as plt

In [ ]:
senses_roots = set('''سمع
بصر
لمس 
شمم
ذوق'''.split('\n'))

## Read in csv

In [ ]:
df_total = pd.read_csv('/media/sf_VBox_Shared/Arabic/Analyses/senses_fiqh.csv')
df_agg_merged = pd.read_csv('/media/sf_VBox_Shared/Arabic/Analyses/senses_fiqh_agg.csv')

In [ ]:
df_total.shape

In [ ]:
df_total.head()

In [ ]:
df_total.root.value_counts()

In [ ]:
df_total.word.value_counts().head(20)

In [ ]:
# For each of the words, what is the number of roots?
df_total.groupby('word').nunique()['root'].sort_values(ascending=False).head()

In [ ]:
df_agg_merged.head()

In [ ]:
df_agg_merged.shape

In [ ]:
import numpy as np
def cast_year(val):
    try:
        return int(str(val).split('-')[0])
    except (ValueError, TypeError):
        return np.nan


df_agg_merged['AuthorDIEDC_int'] = df_agg_merged.AuthorDIEDC.apply(cast_year)

In [ ]:
df_agg_merged[senses_cols].max()

In [ ]:
fig, axes = plt.subplots(1)
colors = ['lightgreen', 'yellow', 'black', 'red']
for i in range(len(senses_cols)):
    df_agg_merged.plot('AuthorDIEDC_int', senses_cols[i], kind='scatter', ax=axes, c=colors[i], label=senses_cols[i])

axes.set_ylim(0, 0.0022)
plt.legend()
plt.show()

In [ ]:
df_agg_merged.AuthorGeographicalArea.value_counts()

In [ ]:
for col in senses_cols:
    ax = df_agg_merged.boxplot(col, by='AuthorGeographicalArea')
    ax.set_title(col)
    plt.show()

In [ ]:
# UTF encoding stuff
arab_text = 'سمع'
latin_text = 'smE'
ltr_char = '\u200E'
rtl_char = '\u061C'
text1 = '{}: ({})'.format(arab_text, latin_text)
text2 = '{}{}{}: ({})'.format(rtl_char, arab_text, ltr_char, latin_text)

In [ ]:
print(text1)
print(text2)

In [ ]:
fig, axes = plt.subplots(2,2)
axes[0,0].set_title(latin_text)
axes[1,0].set_title(arab_text)
axes[0,1].set_title(text1)
axes[1,1].set_title(text2)

for i in range(2):
    for j in range(2):
        axes[i,j].set_xticks([])
plt.show()

In [ ]:
ax